<a href="https://colab.research.google.com/github/arhtudormorar/ML/blob/main/ADX_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read data

In [4]:
import pandas as pd

# Simple CSV loading
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/USDCAD/USDCAD_15min_20241112_to_20241113_with_EMA.csv')
data.head()

,time,Open,High,Low,Close,Volume,EMA_20
0,2024-11-12 00:00:00,1.39182,1.39238,1.39141,1.39193,0,1.391930
1,2024-11-12 00:15:00,1.39193,1.39242,1.39173,1.39233,0,1.391968
2,2024-11-12 00:30:00,1.39233,1.39252,1.39213,1.39231,0,1.392001
3,2024-11-12 00:45:00,1.39229,1.39235,1.39209,1.39233,0,1.392032
4,2024-11-12 01:00:00,1.39226,1.39265,1.39222,1.39259,0,1.392085


In [9]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=e448d8a873a368bb645ff638d0a774f0f58262b1f8335352394e45e5d9d1865a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [10]:
import pandas_ta as ta
data.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
157,14.802249,21.726379,24.547351
158,16.760355,16.814393,41.382139
159,18.193977,16.725649,36.229316
160,18.108402,22.417021,31.597258


Plot data

In [12]:
import plotly.graph_objects as go
import plotly.subplots as ms

dfpl = data[:]
fig = ms.make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.7, 0.3]
)

# Candlestick Chart (Top Panel)
fig.add_trace(
    go.Candlestick(
        x=dfpl.index,
        open=dfpl['Open'],
        high=dfpl['High'],
        low=dfpl['Low'],
        close=dfpl['Close']
    ),
    row=1, col=1
)

# EMA Overlay
fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=dfpl['EMA_20'],
        mode="markers",
        marker=dict(size=5, color="MediumPurple"),
        name="EMA 20"
    ),
    row=1, col=1
)

# ADX Line Chart (Bottom Panel)
fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=dfpl['ADX_14'],
        mode='lines',
        name='ADX',
        line=dict(color='green', width=2)
    ),
    row=2, col=1
)

# Horizontal Lines for ADX
fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=[25]*len(dfpl),
        mode='lines',
        name='ADX 25 Level',
        line=dict(color='red', width=1, dash='dash')
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=[20]*len(dfpl),
        mode='lines',
        name='ADX 20 Level',
        line=dict(color='blue', width=1, dash='dot')
    ),
    row=2, col=1
)

# Layout Customization
fig.update_layout(
    title='Candlestick Chart with EMA and ADX',
    xaxis_rangeslider_visible=False,
    height=800
)

fig.show()


In [13]:
import plotly.graph_objects as go
import plotly.subplots as ms
import numpy as np

dfpl = data[:]

# Initialize variables
rectangles = []
start_index = None
below_25 = False
below_20 = False

# Iterate through ADX values to find start and end points for rectangles
for i in range(len(dfpl['ADX_14'])):
    if dfpl['ADX_14'][i] < 25:
        below_25 = True
        if dfpl['ADX_14'][i] < 20 and below_25:
            if start_index is None:
                start_index = i
                below_20 = True

    if below_20 and dfpl['ADX_14'][i] > 20:
        # Rectangle ends when ADX crosses above 20
        end_index = i
        rectangles.append((start_index, end_index))
        start_index = None
        below_25 = False
        below_20 = False

    if dfpl['ADX_14'][i] > 25:
        # Invalidate start condition if ADX crosses above 25
        start_index = None
        below_25 = False
        below_20 = False

# Calculate max and min prices for each rectangle
rectangle_data = []
for start, end in rectangles:
    max_price = dfpl['High'][start:end+1].max()
    min_price = dfpl['Low'][start:end+1].min()
    rectangle_data.append({
        'start': start,
        'end': end,
        'max_price': max_price,
        'min_price': min_price
    })

# Create Subplot Figure
fig = ms.make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.7, 0.3]
)

# Candlestick Chart (Top Panel)
fig.add_trace(
    go.Candlestick(
        x=dfpl.index,
        open=dfpl['Open'],
        high=dfpl['High'],
        low=dfpl['Low'],
        close=dfpl['Close']
    ),
    row=1, col=1
)

# EMA Overlay
fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=dfpl['EMA_20'],
        mode="markers",
        marker=dict(size=5, color="MediumPurple"),
        name="EMA 20"
    ),
    row=1, col=1
)

# Add Rectangles to the Chart
for rect in rectangle_data:
    start = rect['start']
    end = rect['end']
    max_price = rect['max_price']
    min_price = rect['min_price']

    fig.add_shape(
        type="rect",
        x0=dfpl.index[start],
        x1=dfpl.index[end],
        y0=min_price,
        y1=max_price,
        line=dict(color="RoyalBlue"),
        fillcolor="LightSkyBlue",
        opacity=0.4,
        row=1, col=1
    )

# ADX Line Chart (Bottom Panel)
fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=dfpl['ADX_14'],
        mode='lines',
        name='ADX',
        line=dict(color='green', width=2)
    ),
    row=2, col=1
)

# Horizontal Lines for ADX
fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=[25]*len(dfpl),
        mode='lines',
        name='ADX 25 Level',
        line=dict(color='red', width=1, dash='dash')
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=dfpl.index,
        y=[20]*len(dfpl),
        mode='lines',
        name='ADX 20 Level',
        line=dict(color='blue', width=1, dash='dot')
    ),
    row=2, col=1
)

# Layout Customization
fig.update_layout(
    title='Candlestick Chart with EMA and ADX',
    xaxis_rangeslider_visible=False,
    height=800
)

fig.show()
